## AUTOGRAD: AUTOMATIC DIFFERENTIATION
The ```autograd``` package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

### Backpropagation
Andrej Karpathy's lecture: Good way to understand backpropagation
https://www.youtube.com/watch?v=59Hbtz7XgjM

How jacobian comes into picture? Derivative notes from Justin Johnson
http://cs231n.stanford.edu/handouts/derivatives.pdf

## Tensor
```torch.Tensor``` is the central class of the package. If you set its attribute ```.requires_grad``` as ```True```, it starts to track all operations on it. When you finish your computation you can call ```.backward()``` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into ```.grad``` attribute.

To stop a tensor from tracking history, you can call ```.detach()``` to detach it from the computation history, and to prevent future computation from being tracked.

To prevent tracking history (and using memory), you can also wrap the code block in ```with torch.no_grad():```. This can be particularly helpful when evaluating a model because the model may have trainable parameters with ```requires_grad=True```, but for which we don’t need the gradients.

There’s one more class which is very important for autograd implementation - a ```Function```.

```Tensor``` and ```Function``` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a ```.grad_fn``` attribute that references a ```Function``` that has created the ```Tensor``` (except for Tensors created by the user - their grad_fn is None).

If you want to compute the derivatives, you can call ```.backward()``` on a ```Tensor```. If ```Tensor``` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to ```backward()```, however if it has more elements, you need to specify a ```gradient``` argument that is a tensor of matching shape.

In [1]:
import torch

### Following example in Andrej's lecture (5:20 - 12:00) 

In [2]:
x = torch.tensor(-2.0, requires_grad=True)
y = torch.tensor(5.0, requires_grad=True)
z = torch.tensor(-4.0, requires_grad=True)

In [3]:
x

tensor(-2., requires_grad=True)

In [4]:
q=x+y

In [5]:
f=z*q

In [6]:
f.backward()

In [7]:
print(f.grad)
print(q.grad) # Why there is no gradient? because they are not tracked 
## https://discuss.pytorch.org/t/why-cant-i-see-grad-of-an-intermediate-variable/94

None
None


In [8]:
# d(f)/dx
x.grad

tensor(-4.)

In [9]:
# d(f)/dy
y.grad

tensor(-4.)

In [10]:
# d(f)/dz
z.grad

tensor(3.)

### Following pytorch tutorial

In [12]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [13]:
y=x+2

In [14]:
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


Each tensor has a ```.grad_fn``` attribute that references a ```Function``` that has created the ```Tensor``` (except for Tensors created by the user - their grad_fn is None)

In [15]:
# y was created as a result of an operation, so it has a grad_fn
print(y.grad_fn)

In [16]:
print(x.grad_fn)

None


In [17]:
z = y * y * 3
out = z.mean()

In [18]:
print(z)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


In [19]:
print(out)

tensor(27., grad_fn=<MeanBackward0>)


```.requires_grad_( ... )``` changes an existing Tensor’s ```requires_grad``` flag in-place. The input flag defaults to False if not given.

In [20]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

False


In [21]:
a.requires_grad_(True)
print(a.requires_grad)

True


In [22]:
b = (a * a).sum()
print(b.grad_fn)

Let’s backprop now. Because out contains a single scalar, ```out.backward()``` is equivalent to ```out.backward(torch.tensor(1.))```.

In [23]:
out.backward()

In [24]:
# Print gradients d(out)/dx
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


#### Another example

In [25]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    print(y)

tensor([ 0.0406, -0.1232, -2.7740], grad_fn=<MulBackward0>)
tensor([ 0.0811, -0.2464, -5.5480], grad_fn=<MulBackward0>)
tensor([  0.1623,  -0.4928, -11.0960], grad_fn=<MulBackward0>)
tensor([  0.3246,  -0.9856, -22.1920], grad_fn=<MulBackward0>)
tensor([  0.6492,  -1.9711, -44.3841], grad_fn=<MulBackward0>)
tensor([  1.2983,  -3.9423, -88.7682], grad_fn=<MulBackward0>)
tensor([   2.5966,   -7.8845, -177.5364], grad_fn=<MulBackward0>)
tensor([   5.1932,  -15.7691, -355.0727], grad_fn=<MulBackward0>)
tensor([  10.3865,  -31.5382, -710.1454], grad_fn=<MulBackward0>)
tensor([   20.7730,   -63.0764, -1420.2909], grad_fn=<MulBackward0>)


In [26]:
# Now in this case y is no longer a scalar. 
# torch.autograd could not compute the full Jacobian directly, 
# but if we just want the vector-Jacobian product, simply pass the vector to backward as argument:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v, retain_graph=True)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


To reduce memory usage, during the .backward() call, all the intermediary results are deleted when they are not needed anymore. Hence if you try to call .backward() again, the intermediary results don’t exist and the backward pass cannot be performed (and you get the error you see).
You can call .backward(retain_graph=True) to make a backward pass that will not delete intermediary results, and so you will be able to call .backward() again. All but the last call to backward should have the retain_graph=True option.

In [27]:
y.backward(torch.ones_like(y), retain_graph=True)

print(x.grad)

tensor([2252.8000, 4096.0000, 2048.2048])


In [28]:
# using no_grad() block to turn off function tracking
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [29]:
# or use .detach()
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
